# Set environment

In [ ]:
# # http://pytorch.org/
# from os.path import exists
# from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
# platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
# cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
# accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

# !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

In [ ]:
# !git clone https://github.com/gilbertolem/ProgGen

In [ ]:
# from os import chdir, getcwd
# chdir("ProgGen")
# from sys import path
# path.append(getcwd())

# Train

In [1]:
import torch
import utils.data_tools as data_tools
from utils.nets import ProgGenRNN, WeightedLoss
from pickle import load
import matplotlib.pyplot as plt
from numpy import argmin

xml_directory = "XML_Tunes/"
torch.manual_seed(999)
use_gpu = torch.cuda.is_available()
print("GPU available: {}".format(use_gpu))

GPU available: False


In [2]:
# Load vocabulary
words_text2num = load(open("maps/words_text2num.txt",'rb'))
vocab_size = len(words_text2num)

# Create training data
filter_names = ['Charlie Parker']
filter_fracs = [1.0]
filters = {'names':filter_names, 'frac':filter_fracs}
Train, Val = data_tools.musicxml2tensor(xml_directory, words_text2num, filters=filters)
train_data = data_tools.TuneData(Train)
val_data = data_tools.TuneData(Val)


CREATING TENSORS FROM MUSICXML FILES...


	297 tunes successfully loaded for training.
	75 tunes successfully loaded for validation.


In [6]:
# Construct Neural Net

# Embedding type
embed_size = 100 

# Net Type
rnn_type = 'lstm'
bidirectional = True

# RNN parameters
num_layers = 1
hidden_rnn = 100
dropout_rnn = 0.0

# FC layers parameters
hidden_fc = 0
dropout_fc = 0.0

# Create model and loss function
model = ProgGenRNN(vocab_size, embed_size, rnn_type, bidirectional, 
                   hidden_rnn, num_layers, dropout_rnn, 
                   hidden_fc, dropout_fc)
loss_fn = WeightedLoss()

# Define loader
sampler = torch.utils.data.RandomSampler(train_data)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=3000 if use_gpu else 512, sampler=sampler, num_workers=1 if use_gpu else 4)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=3000 if use_gpu else 512, num_workers=1 if use_gpu else 4)

if use_gpu:
    model = model.cuda()
    loss_fn = loss_fn.cuda()

In [7]:
# Define loss function and optimizer
lr = 1e-2
optim = torch.optim.Adam(model.parameters(), lr=lr)

In [8]:
from utils.training import train
epochs = 500
losses = train(epochs, model, optim, train_loader, val_loader, loss_fn, use_gpu)

idx = argmin(losses[1])
print("Best Loss:\n\tTrain: {}\n\tVal: {}".format(losses[0][idx], losses[1][idx]))

plt.semilogy(losses[0], label='Train')
plt.semilogy(losses[1], label='Val')
plt.legend()
plt.show()


--------------------------------------------------------------------
TRAINING MODEL... 

   Epoch | Training Loss | Val. Loss


       0 |          5.49 |      5.47


       1 |          3.93 |      3.66


       2 |          3.14 |      2.84


       3 |          3.11 |      2.78


...

      72 |          1.24 |       1.4


      73 |          1.23 |       1.4


      74 |          1.23 |      1.39


      75 |          1.22 |       1.4


      76 |          1.21 |      1.41
Best Loss:
	Train: 1.2481709718704224
	Val: 1.3918592929840088


# Generate something

In [14]:
from utils.generating import generate_progression

model_name = "model"
initial_chord = "4C_m"
tune_len = 32
top = 2

prog = generate_progression(initial_chord, tune_len, top, model_name, verbose = False)
print("Generated Progression:\n")
print(prog)

Generated Progression:

Cm             |F7             |F7             |Bb7            |
Eb7            |Eb7            |Abmaj          |Gm7b5   C7alt  |
Fm      Bb7    |Ebmaj          |Ebmaj          |Fm      Bb7alt |
Ebm     Ab7    |C#m     F#7    |Bmaj           |C#m     F#7    |
Bm      E7     |Amaj           |Amaj           |Amaj           |
Amaj           |Amaj           |Amaj           |Amaj           |
Amaj           |Amaj           |
